In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Guardar transformers
import pickle


# Preprocesado
from sklearn.preprocessing import RobustScaler


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df_casual = pd.read_pickle('../../../datos_finales/archivo_casual_est_enc2.pkl')
df_casual.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,usuarios_casuales
0,1.0,1.0,1.0,2.67,1.79,4.25,-0.486274,0.855582,-0.208909,331
1,1.0,1.0,1.0,1.83,1.00,4.25,-0.425690,0.332722,0.687550,131


In [3]:
df_casual_limpio = pd.read_pickle('../../../datos_finales/archivo_casual_est.pkl')
df_casual_limpio.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,usuarios_casuales
0,invierno,2018,Enero,Lunes,festivo,nuboso,-0.486274,0.855582,-0.208909,331
1,invierno,2018,Enero,Martes,laborable,nuboso,-0.425690,0.332722,0.687550,131


### Inputs

In [7]:
estacion = input('Especifica la estación').lower()
año = int(input('Especifica el año'))
mes = input('Especifica el mes').lower().capitalize()
dia_semana = input('Especifica el dia de la semana').lower().capitalize()
dias_laborables = input('Especifica si es día es laborable/festivo/fin de semana').lower()
tiempo = input('Especifica si es día es nuboso/despejado/lluvioso').lower()
temperatura = int(input('Especifica la temperatura'))
humedad = int(input('Especifica la humedad'))
velocidad_viento = int(input('Especifica la velocidad del viento'))

In [13]:
diccionario_aut = {}

In [14]:
for col in df_casual.columns[:-1]:
    diccionario_aut[col] = globals()[col]

In [19]:
diccionario_aut

{'estacion': 'otoño',
 'año': 2019,
 'mes': 'Octubre',
 'dia_semana': 'Martes',
 'dias_laborables': 'laborable',
 'tiempo': 'despejado',
 'temperatura': 25,
 'humedad': 20,
 'velocidad_viento': 5}

In [30]:
usuario = pd.DataFrame(diccionario_aut,index=[0])

In [31]:
usuario

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento
0,otoño,2019,Octubre,Martes,laborable,despejado,25,20,5


### Estandarización

In [33]:
with open("../../../datos_finales/predicciones/estandarizacion.pkl", "rb") as est:
    estandarizacion = pickle.load(est)

In [36]:
numericas = usuario.select_dtypes(include=np.number).drop('año',axis=1)
numericas

,temperatura,humedad,velocidad_viento
0,25,20,5


In [37]:
numericas_robust = pd.DataFrame(estandarizacion.transform(numericas), columns = numericas.columns)
numericas_robust.head(2)

,temperatura,humedad,velocidad_viento
0,0.346948,-2.03077,-1.082228


In [38]:
usuario[numericas_robust.columns] = numericas_robust
usuario

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento
0,otoño,2019,Octubre,Martes,laborable,despejado,0.346948,-2.03077,-1.082228


### Encoding

In [40]:
usuario.loc[0,'año'] = 1.47
usuario

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento
0,otoño,1.47,Octubre,Martes,laborable,despejado,0.346948,-2.03077,-1.082228


In [43]:
def encoding(dataframe,dataframe_final, columnas,variable_respuesta):
    """
    Esta función realiza el encoding de manera automática teniendo en cuenta las medias de los valores únicos de todas las variables categóricas.
    Args:
        dataframe (dataframe): el dataframe donde se encuentran las columnas categóricas.
        columnas (category): las columnas categóricas sobre las cuales queremos hacder el encoding.
        variable_respuesta (str): nuestra variable respuesta la cual utilizamos para saber la mediana.
    Retuns: None
    """
    for col in columnas:
        diccionario = {}
        df_mediana = dataframe.groupby(col)[variable_respuesta].median().reset_index().sort_values(by=variable_respuesta)
        df_mediana['encoding'] = round(df_mediana[variable_respuesta]/df_mediana.iloc[0,1], 2)
        for indice in range(df_mediana.shape[0]):
            diccionario[df_mediana.iloc[indice,0]] = df_mediana.iloc[indice,2]
        dataframe_final[col] = dataframe_final[col].map(diccionario)

In [47]:
categoricas = df_casual_limpio.select_dtypes(include='O')
categoricas

,estacion,mes,dia_semana,dias_laborables,tiempo
0,invierno,Enero,Lunes,festivo,nuboso
1,invierno,Enero,Martes,laborable,nuboso
2,invierno,Enero,Miercoles,laborable,despejado
3,invierno,Enero,Jueves,laborable,despejado
4,invierno,Enero,Viernes,laborable,despejado
...,...,...,...,...,...
725,invierno,Diciembre,Viernes,laborable,nuboso
726,invierno,Diciembre,Domingo,fin de semana,nuboso
727,invierno,Diciembre,Sabado,fin de semana,nuboso
728,invierno,Diciembre,Lunes,laborable,despejado


In [48]:
encoding(df_casual_limpio,usuario, categoricas.columns,'usuarios_casuales')

In [49]:
usuario

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento
0,2.48,1.47,6.1,1.83,1.0,6.58,0.346948,-2.03077,-1.082228


### Predicción

In [52]:
with open("../../../datos_finales/predicciones/mejor_modelo_causal.pkl", "rb") as modelo:
    mejor_modelo = pickle.load(modelo)

In [54]:
mejor_modelo.predict(usuario)[0]

1675.2035587939183

In [56]:
metricas = pd.read_pickle("../../../datos_finales/predicciones/metricas_casuales.pkl")
metricas

,MAE,MSE,RMSE,R2,set,modelo
0,246.577182,122397.720814,349.853856,0.74523,test,Random Forest
1,236.504806,119807.611363,346.132361,0.74384,train,Random Forest


In [57]:
metricas.loc[0,'RMSE']

349.8538563664424

In [61]:
print(f'La predicciones de bicicletas alquiladas para ese día por los clientes casuales es entre {round(mejor_modelo.predict(usuario)[0] + metricas.loc[0,"RMSE"],0)} y {round(mejor_modelo.predict(usuario)[0] - metricas.loc[0,"RMSE"],0)}')

La predicciones de bicicletas alquiladas para ese día por los cliente casuales es entre 2025.0 y 1325.0
